In [1]:
import random
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

class PDPTWInstance:
    def __init__(self, n, map_size, speed, extra_time,seed=None):
        """
        初始化 PDPTW 实例
        :param n: pickup 点的数量
        :param map_size: 地图大小
        :param speed: 车辆速度
        :param extra_time: delivery 点时间窗口起始时间的额外时间
        :param seed: 随机数种子
        """
        self.n = n
        self.map_size = map_size
        self.speed = speed
        self.extra_time = extra_time
        # coordinates
        self.depot = (0, 0)  # depot 位于原点
        self.pickup_points = []  # pickup 点的坐标
        self.delivery_points = []  # delivery 点的坐标
        # time
        self.time_windows = []  # 时间窗口列表
        self.service_times = []  # 服务时间列表
        # demand
        self.demands = []  # 需求量列表

        # 设置随机数种子
        if seed is not None:
            random.seed(seed)
            np.random.seed(seed)
        
        # 生成所有点的索引列表
        self.indices = [0] + list(range(1, self.n+1)) + list(range(self.n+1, 2*self.n+1))
        
        self.generate_points()  # 生成 pickup 和 delivery 点
        self.generate_time_windows()  # 生成时间窗口和服务时间
        self.generate_demands()  # 生成需求量

    def generate_points(self):
        """
        生成 pickup 和 delivery 点
        """
        for _ in range(self.n):
            # 在地图范围内随机生成 pickup 点坐标
            pickup_x = random.uniform(-self.map_size, self.map_size)
            pickup_y = random.uniform(-self.map_size, self.map_size)
            self.pickup_points.append((pickup_x, pickup_y))

            # 在地图范围内随机生成 delivery 点坐标
            delivery_x = random.uniform(-self.map_size, self.map_size)
            delivery_y = random.uniform(-self.map_size, self.map_size)
            self.delivery_points.append((delivery_x, delivery_y))

    def plot_instance(self):
        """
        绘制 PDPTW 实例图
        """
        plt.figure(figsize=(8, 8))
        # 绘制 depot
        plt.scatter(self.depot[0], self.depot[1], c='red', marker='s', s=100, label='Depot')
        # 绘制 pickup 点
        plt.scatter([p[0] for p in self.pickup_points], [p[1] for p in self.pickup_points], c='blue', marker='o', label='Pickup')
        # 绘制 delivery 点
        plt.scatter([d[0] for d in self.delivery_points], [d[1] for d in self.delivery_points], c='green', marker='d', label='Delivery')
        
        # 为每个点添加标签
        for i in range(self.n):
            plt.annotate(f'P{i+1}', (self.pickup_points[i][0], self.pickup_points[i][1]), textcoords='offset points', xytext=(0,5), ha='center')
            plt.annotate(f'D{i+1}', (self.delivery_points[i][0], self.delivery_points[i][1]), textcoords='offset points', xytext=(0,5), ha='center')

        # 设置坐标轴范围
        plt.xlim(-self.map_size-1, self.map_size+1)
        plt.ylim(-self.map_size-1, self.map_size+1)
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.title('PDPTW Instance')
        plt.legend()
        plt.grid(True)
        plt.show()
        
    def calculate_distance_matrix(self):
        """
        计算距离矩阵
        :return: 距离矩阵
        """
        points = [self.depot] + self.pickup_points + self.delivery_points
        num_points = len(points)
        distance_matrix = np.zeros((num_points, num_points))
        
        # 计算每对点之间的欧几里得距离
        for i in range(num_points):
            for j in range(num_points):
                distance_matrix[i][j] = np.sqrt((points[i][0] - points[j][0])**2 + (points[i][1] - points[j][1])**2)
        
        return distance_matrix
    
    def calculate_time_matrix(self):
        """
        计算时间矩阵
        :return: 时间矩阵
        """
        distance_matrix = self.calculate_distance_matrix()
        time_matrix = (distance_matrix / self.speed)*60
        return time_matrix
    
    def generate_time_windows(self):
        """
        生成时间窗口和服务时间
        """
        time_matrix = self.calculate_time_matrix()
        
        # Depot 的时间窗口设置为 [0, inf]
        self.time_windows.append((0, float('inf')))
        
        for i in range(self.n):
            # Pickup 点的起始时间在 [0, 120] 范围内随机生成，结束时间设置为 inf
            pickup_start_time = random.randint(0, 120)
            self.time_windows.append( (pickup_start_time, float('inf')) )
            
            # 每个 pickup 点和 delivery 点的服务时间在 [5, 10] 范围内随机生成一个整数
            self.service_times.append(random.randint(5, 10))
        
        for i in range(1,self.n + 1):
            pickup_start_time = self.time_windows[i][0]
            # Delivery 点的起始时间为对应 pickup 点的起始时间 + 时间矩阵中的时间 + 额外时间，向上取整
            # 结束时间为起始时间 + 30 分钟，向下取整
            delivery_start_time = math.ceil(pickup_start_time + time_matrix[i][i+self.n] + self.extra_time)
            delivery_end_time = math.floor(delivery_start_time + 60)
            self.time_windows.append((delivery_start_time, delivery_end_time))
            
            # 每个 pickup 点和 delivery 点的服务时间在 [5, 10] 范围内随机生成一个整数
            self.service_times.append(random.randint(5, 10))
        
        # Depot 的服务时间设置为 0
        self.service_times.append(0)
        
    def generate_demands(self):
        """
        生成需求量
        """
        # Depot 的需求量为 0
        self.demands.append(0)
        
        for _ in range(self.n):
            # Pickup 点的需求量为 1
            self.demands.append(1)
        for _ in range(self.n):
            # Delivery 点的需求量为 -1
            self.demands.append(-1)
            
    def to_dataframe(self):
        """
        将 PDPTW 实例转换为 pandas 数据框
        :return: pandas 数据框
        """
        data = []
        
        for i in range(len(self.indices)):
            if i == 0:
                point_type = 'd'
                x, y = self.depot
                partner_id = 0
            elif i <= self.n:
                point_type = 'cp'
                x, y = self.pickup_points[i-1]
                partner_id = i + self.n
            else:
                point_type = 'cd'
                x, y = self.delivery_points[i-self.n-1]
                partner_id = i - self.n
                
            data.append([
                i,
                point_type,
                x,
                y,
                self.demands[i],
                self.time_windows[i][0],
                self.time_windows[i][1],
                self.service_times[i],
                partner_id
            ])
        
        columns = ['ID', 'Type', 'X', 'Y', 'Demand', 'ReadyTime', 'DueDate', 'ServiceTime', 'PartnerID']
        df = pd.DataFrame(data, columns=columns)
        
        return df

In [2]:
import numpy as np
import matplotlib.pyplot as plt

class PDPTWSolution:
    def __init__(self, instance, vehicle_capacity, battery_capacity, battery_consume_rate, routes):
        """
        初始化 PDPTWSolution 对象
        :param instance: PDPTWInstance 对象
        :param vehicle_capacity: 车辆容量
        :param battery_capacity: 电池容量
        :param battery_consume_rate: 电池消耗率
        :param routes: 给定的路径列表
        """
        self.instance = instance
        self.vehicle_capacity = vehicle_capacity
        self.battery_capacity = battery_capacity
        self.battery_consume_rate = battery_consume_rate
        self.routes = routes
        self.num_vehicles = len(routes)
        # battery and capacity
        self.route_battery_levels = np.empty((self.num_vehicles,), dtype=object)
        self.route_capacity_levels = np.empty((self.num_vehicles,), dtype=object)
        # time
        self.route_arrival_times = np.empty((self.num_vehicles,), dtype=object)
        self.route_leave_times = np.empty((self.num_vehicles,), dtype=object)
        self.route_wait_times = np.empty((self.num_vehicles,), dtype=object)
        # evaluation
        self.total_travel_times = np.zeros((self.num_vehicles,))
        self.total_delay_times = np.zeros((self.num_vehicles,))
        self.total_wait_times = np.zeros((self.num_vehicles,))

        self.initialize_routes()
        self.calculate_all()

    def initialize_routes(self):
        """
        初始化与路径相关的变量
        """
        for vehicle_id in range(self.num_vehicles):
            route = self.routes[vehicle_id]
            self.route_battery_levels[vehicle_id] = np.zeros((len(route),))
            self.route_capacity_levels[vehicle_id] = np.zeros((len(route),))
            self.route_arrival_times[vehicle_id] = np.zeros((len(route),))
            self.route_leave_times[vehicle_id] = np.zeros((len(route),))
            self.route_wait_times[vehicle_id] = np.zeros((len(route),))

    def calculate_all(self):
        """
        计算所有状态变量和目标函数值
        """
        for vehicle_id in range(self.num_vehicles):
            self.calculate_battery_capacity_levels(vehicle_id)
            self.calculate_arrival_leave_wait_times(vehicle_id)
            self.calculate_travel_delay_wait_times(vehicle_id)

    def calculate_battery_capacity_levels(self, vehicle_id):
        """
        计算指定车辆在路径上每个节点的电池电量和容量水平
        :param vehicle_id: 车辆 ID
        """
        route = self.routes[vehicle_id]
        battery_levels = self.route_battery_levels[vehicle_id]
        capacity_levels = self.route_capacity_levels[vehicle_id]
        time_matrix = self.instance.calculate_time_matrix()

        battery_levels[0] = self.battery_capacity
        capacity_levels[0] = 0

        for i in range(1, len(route)):
            prev_node = route[i - 1]
            curr_node = route[i]
            # update the battery
            time = time_matrix[prev_node][curr_node]
            battery_levels[i] = battery_levels[i - 1] - time * self.battery_consume_rate
            # update the capacity
            capacity_levels[i] = capacity_levels[i - 1] + self.instance.demands[curr_node]

    def calculate_arrival_leave_wait_times(self, vehicle_id):
        """
        计算指定车辆在路径上每个节点的到达时间、离开时间和等待时间
        :param vehicle_id: 车辆 ID
        """
        route = self.routes[vehicle_id]
        arrival_times = self.route_arrival_times[vehicle_id]
        leave_times = self.route_leave_times[vehicle_id]
        wait_times = self.route_wait_times[vehicle_id]
        time_matrix = self.instance.calculate_time_matrix()

        arrival_times[0] = 0
        wait_times[0] = 0
        leave_times[0] = self.instance.service_times[route[0]]

        for i in range(1, len(route)):
            prev_node = route[i - 1]
            curr_node = route[i]
            arrival_times[i] = leave_times[i - 1] + time_matrix[prev_node][curr_node]
            wait_times[i] = max(0, self.instance.time_windows[curr_node][0] - arrival_times[i])
            if arrival_times[i] > self.instance.time_windows[curr_node][0]:
                wait_times[i] = 0
            leave_times[i] = arrival_times[i] + wait_times[i] + self.instance.service_times[curr_node]

    def calculate_travel_delay_wait_times(self, vehicle_id):
        """
        计算指定车辆的总行驶时间、总延误时间和总等待时间
        :param vehicle_id: 车辆 ID
        """
        route = self.routes[vehicle_id]
        leave_times = self.route_leave_times[vehicle_id]
        wait_times = self.route_wait_times[vehicle_id]
        time_matrix = self.instance.calculate_time_matrix()

        travel_time = 0
        delay_time = 0
        wait_time = 0

        for i in range(len(route) - 1):
            curr_node = route[i]
            next_node = route[i + 1]
            travel_time += time_matrix[curr_node][next_node]
            
            if self.instance.time_windows[next_node][1] != float('inf'):
                delay_time += max(0, leave_times[i] - self.instance.time_windows[next_node][1])
            
            wait_time += wait_times[i]

        self.total_travel_times[vehicle_id] = travel_time
        self.total_delay_times[vehicle_id] = delay_time
        self.total_wait_times[vehicle_id] = wait_time

    def objective_function(self):
        """
        计算目标函数值
        :return: 目标函数值（总行驶时间 + 总延误时间）
        """
        return np.sum(self.total_travel_times) + np.sum(self.total_delay_times)

    def is_feasible(self):
        """
        检查解的可行性
        """
        selected_vehicles = self.get_selected_vehicles()
        if not selected_vehicles:
            return False
        if not self.check_capacity_constraint(selected_vehicles):
            return False
        if not self.check_battery_constraint(selected_vehicles):
            return False
        # if not self.check_unique_service(selected_vehicles):
        #     return False
        # if not self.check_depot_constraint(selected_vehicles):
        #     return False
        # if not self.check_pickup_delivery_order(selected_vehicles):
        #     return False
        return True

    def get_selected_vehicles(self):
        """
        获取被选择的车辆列表
        :return: 被选择的车辆 ID 列表
        """
        selected_vehicles = []
        for vehicle_id in range(self.num_vehicles):
            if self.routes[vehicle_id] is not None and len(self.routes[vehicle_id]) > 2:
                selected_vehicles.append(vehicle_id)
        return selected_vehicles

    def check_capacity_constraint(self, selected_vehicles):
        """
        检查容量约束
        :param selected_vehicles: 被选择的车辆 ID 列表
        """
        for vehicle_id in selected_vehicles:
            if np.any(self.route_capacity_levels[vehicle_id] > self.vehicle_capacity):
                return False
        return True

    def check_battery_constraint(self, selected_vehicles):
        """
        检查电池约束
        :param selected_vehicles: 被选择的车辆 ID 列表
        """
        for vehicle_id in selected_vehicles:
            if np.any(self.route_battery_levels[vehicle_id] < 0):
                return False
        return True

    def check_unique_service(self, selected_vehicles):
        """
        检查唯一服务约束
        :param selected_vehicles: 被选择的车辆 ID 列表
        """
        visited_nodes = set()
        for vehicle_id in selected_vehicles:
            route = self.routes[vehicle_id]
            for node in route:
                if node in visited_nodes:
                    return False
                visited_nodes.add(node)
        return True

    def check_depot_constraint(self, selected_vehicles):
        """
        检查车辆是否从车场出发并返回车场
        :param selected_vehicles: 被选择的车辆 ID 列表
        """
        for vehicle_id in selected_vehicles:
            route = self.routes[vehicle_id]
            if route[0] != 0 or route[-1] != 0:
                return False
        return True

    def check_pickup_delivery_order(self, selected_vehicles):
        """
        检查取货-送货顺序约束
        :param selected_vehicles: 被选择的车辆 ID 列表
        :return: True 如果取货-送货顺序约束满足，False 否则
        """
        for vehicle_id in selected_vehicles:
            route = self.routes[vehicle_id]
            visited_pickups = set()
            for node in route:
                if 1 <= node <= self.instance.n:
                    visited_pickups.add(node)
                elif self.instance.n < node <= 2 * self.instance.n:
                    if node - self.instance.n not in visited_pickups:
                        return False
        return True

    def plot_solution(self, vehicle_ids=None):
        """
        绘制解的路线图
        :param vehicle_ids: 要绘制路径的车辆 ID 列表（可选），默认为 None，表示绘制所有被选择车辆的路径
        """
        selected_vehicles = self.get_selected_vehicles()
        if not selected_vehicles:
            print("No vehicle is selected in the solution.")
            return
    
        if vehicle_ids is None:
            vehicle_ids = selected_vehicles
        else:
            vehicle_ids = [vehicle_id for vehicle_id in vehicle_ids if vehicle_id in selected_vehicles]
    
        if not vehicle_ids:
            print("No valid vehicle IDs provided.")
            return
    
        plt.figure(figsize=(8, 8))
        plt.scatter(self.instance.depot[0], self.instance.depot[1], c='red', marker='s', s=100, label='Depot')
        plt.scatter([p[0] for p in self.instance.pickup_points], [p[1] for p in self.instance.pickup_points], c='blue', marker='o', label='Pickup')
        plt.scatter([d[0] for d in self.instance.delivery_points], [d[1] for d in self.instance.delivery_points], c='green', marker='d', label='Delivery')
    
        plt.annotate('Depot', (self.instance.depot[0], self.instance.depot[1]), textcoords='offset points', xytext=(0, 10), ha='center')
        for i, pickup_point in enumerate(self.instance.pickup_points, start=1):
            plt.annotate(f'P{i}', (pickup_point[0], pickup_point[1]), textcoords='offset points', xytext=(0, 5), ha='center')
        for i, delivery_point in enumerate(self.instance.delivery_points, start=1):
            plt.annotate(f'D{i}', (delivery_point[0], delivery_point[1]), textcoords='offset points', xytext=(0, 5), ha='center')
    
        color_map = plt.cm.get_cmap('viridis', len(vehicle_ids))
        for i, vehicle_id in enumerate(vehicle_ids):
            route = self.routes[vehicle_id]
            color = color_map(i)
            
            route_points = [self.instance.depot] + [self.instance.pickup_points[node-1] if node <= self.instance.n else self.instance.delivery_points[node-self.instance.n-1] for node in route[1:-1]] + [self.instance.depot]
            route_x = [point[0] for point in route_points]
            route_y = [point[1] for point in route_points]
            
            plt.plot(route_x, route_y, color=color, linestyle='-', linewidth=1.5, alpha=0.8, label=f'Vehicle {vehicle_id+1}')
    
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.title('PDPTW Solution')
        plt.legend()
        plt.grid(True)
        plt.show()

In [18]:
import pandas as pd
import numpy as np
import copy

# Function to calculate Euclidean distance
def euclidean_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

# Insertion heuristic algorithm
def insert_request(routes, pickup_delivery_pairs, locations, earliest, latest, demand, penalty, total_orders, vehicle_capacity, services, battery, speed):
    capacity_dict = {0:0}
    time_dict = {0:0}
    remaining_orders = []
    pickup_delivery_pairs_copy = copy.deepcopy(pickup_delivery_pairs[1:(total_orders+1)])
    # print('pickup_delivery_pairs_copy',pickup_delivery_pairs_copy)
    random.shuffle(pickup_delivery_pairs_copy)
    # print('pickup_delivery_pairs_copy_shuffle',pickup_delivery_pairs_copy)
    
    for request in pickup_delivery_pairs_copy:  #Enumerate number of orders
        # print('request',request[0])
        best_vehicle = None
        best_pickup_position = None
        best_dropoff_position = None
        best_cost = float('inf')
        best_dist_total = float('inf')

        for vehicle_idx, route in enumerate(routes):
            for i in range(1, len(route)):
                # if np.random.uniform(0,1) < 0.99:
                    route_copy = copy.deepcopy(route)
                    new_route = route_copy[:i] + [int(request[0])] + route_copy[i:]
                    for j in range(i+1, len(new_route)):
                        new_route = new_route[:j] + [int(request[0]+total_orders)] + new_route[j:]
                        new_time_dict = copy.deepcopy(time_dict)
                        new_capacity_dict = copy.deepcopy(capacity_dict)
                        new_time_dict, new_capacity_dict = update_node(new_route, earliest, new_time_dict, new_capacity_dict, services, demand, locations, speed)
                        dist = total_distance(new_route, locations)
                        if any(value > vehicle_capacity for value in new_capacity_dict.values()) or (dist > battery):
                            continue
                        else:
                            dist_total = 0
                            new_routes = copy.deepcopy(routes)
                            new_routes[vehicle_idx] = new_route
                            cost = calculate_route_cost(new_routes, new_time_dict, locations, latest, penalty, remaining_orders)
                            for r in new_routes:
                                dist_total += total_distance(r, locations)
                            if (cost <= best_cost) and (dist_total <= best_dist_total*1.01):
                                best_vehicle = vehicle_idx
                                best_pickup_position = i
                                best_dropoff_position = j
                                best_cost = cost
                                if dist_total < best_dist_total:
                                    best_dist_total = dist_total
                            

        if best_vehicle is not None:
            routes[best_vehicle].insert(best_pickup_position, int(request[0]))
            routes[best_vehicle].insert(best_dropoff_position, int(request[0]+total_orders))

            time_dict, capacity_dict = update_node(routes[best_vehicle], earliest, time_dict, capacity_dict, services, demand, locations, speed)
        else:
            print('Need more vehicles!')
            remaining_orders.append(int(request[0]))
            print('remaining_orders:',remaining_orders)
            print('***************')

    return routes, remaining_orders

def calculate_route_cost(routes, time_dict, locations, latest, penalty, remaining_orders):
    cost = 0
    for vehicle_idx, route in enumerate(routes):
        for i in range(len(route) - 1):
            cost += euclidean_distance(locations[route[i]], locations[route[i+1]])
    for j in time_dict:
        if time_dict[j] > latest[j]:
            cost += penalty  #penalty cost
    cost = cost + len(remaining_orders)*50
    return cost

def update_node(route, earliest, time_dict, capacity_dict, services, demand, locations, speed):
    for idx, node in enumerate(route[:-1]):
        if idx == 0:
            continue
        time_dict[node] = max(earliest[node],
                                time_dict[route[idx-1]] + services[route[idx-1]] + euclidean_distance(locations[route[idx-1]], locations[route[idx]])/speed*60)[0] #speed is 4
        capacity_dict[node] = (capacity_dict[route[idx-1]] + demand[node])[0]
    return time_dict, capacity_dict

def total_distance(route, locations):
    dist = 0
    for i in range(len(route) - 1):
            dist += euclidean_distance(locations[route[i]], locations[route[i+1]])
    return dist

# Execute the insertion heuristic
def insertion_heuristics(df, routes=None, pickup_delivery_pairs=None, num_vehicles=None, vehicle_capacity=None, penalty=None, speed=None, battery=None):
    
    orders = df
    total_orders = int(orders.shape[0]/2-1)

    if num_vehicles == None:
        num_vehicles = 5
    else:
        num_vehicles = num_vehicles
    
    if vehicle_capacity == None:
        vehicle_capacity = 6
    else:
        vehicle_capacity = vehicle_capacity

    if penalty == None:
        penalty = 2
    else:
        penalty = penalty
    
    if speed == None:
        speed = 4
    else:
        speed = speed

    if battery == None:
        battery = 20
    else:
        battery = battery
    # service = 2

    # Extract necessary information from the DataFrame
    if pickup_delivery_pairs == None:
        pickup_delivery_pairs = orders[['ID']].values.tolist()
    else:
        pickup_delivery_pairs = pickup_delivery_pairs
    locations = orders[['X','Y']].values
    earliest = orders[['ReadyTime']].values
    latest = orders[['DueDate']].values
    demand = orders[['Demand']].values
    services = orders[['ServiceTime']].values

    # Initialize the routes, start and end with depot
    if routes == None:
        routes = [[0, 0] for _ in range(num_vehicles)]
    else:
        routes = routes


    routes, remaining_orders = insert_request(routes, pickup_delivery_pairs, locations, earliest, latest, demand, penalty, total_orders, vehicle_capacity, services, battery, speed)

    return routes, remaining_orders

In [19]:
import random
import numpy as np
import matplotlib.pyplot as plt

# 创建一个 PDPTWInstance 对象
instance = PDPTWInstance(n=15, map_size=2, speed=4, extra_time=0,seed = 1234)
df = instance.to_dataframe()
d_last = []
d_last.append([int(max(df.ID)+1),'d',df.X[0],df.Y[0],0,0,float('inf'),0,0])
columns = ['ID', 'Type', 'X', 'Y', 'Demand', 'ReadyTime', 'DueDate', 'ServiceTime', 'PartnerID']
d_last = pd.DataFrame(d_last, columns=columns)
df = pd.concat([df, d_last], axis = 0)

seeding = 2
random.seed(seeding)
np.random.seed(seeding)

routes = insertion_heuristics(df, battery = 25,num_vehicles = 4)
routes

([[0, 0],
  [0, 8, 23, 0],
  [0, 4, 19, 7, 22, 11, 26, 12, 9, 24, 14, 1, 27, 16, 15, 30, 29, 0],
  [0, 13, 2, 3, 6, 5, 17, 20, 18, 21, 10, 25, 28, 0]],
 [])

### SISR process

In [20]:
import warnings

# Generate number of strings to remove
def number_of_strings(L_max, avg_remove_order, routes, n_orders):
    T_avg = np.floor(n_orders/len(routes))
    l_s_max = min(T_avg, L_max)
    k_s_max = 4*avg_remove_order/(1+l_s_max) - 1
    k_s = np.floor(np.random.uniform(1, k_s_max + 1))
    
    return k_s, l_s_max

# Generate number of orders to remove in each string
def number_of_orders(l_s_max, route):
    l_t_max = min(l_s_max, (len(route)-2)/2)
    l_t = np.floor(np.random.uniform(1, l_t_max + 1))
    
    return l_t

# Generate random order to start, remove orders according to order distance matrix
def order_to_start(n_orders, remaining_order = None):
    reference_order = np.floor(np.random.uniform(1, n_orders + 1))
    if remaining_order != None:
        while reference_order in remaining_order:
            reference_order = np.floor(np.random.uniform(1, n_orders + 1))
    return reference_order

def find_lists_containing_element(routes, order):
    return [route for route in routes if order in route][0]

def primal_string_removal(d_matrix, routes, route, l_t, reference_order, n_orders, removed_list, deconstructed_route_list):
    distances = d_matrix[reference_order-1]
    sorted_indices = np.argsort(distances).tolist()
    # print('sorted_indices',sorted_indices)
    # sorted_indices_updated = sorted_indices.copy()
    route_1 = copy.deepcopy(route)
    a = 0
    for i in sorted_indices:
        if a <= l_t-1:
            if i+1 in route:
                route_1.remove(i+1)
                removed_list.append(i+1)
                route_1.remove(i+1+n_orders)
                removed_list.append(i+1+n_orders)
                a += 1
        else:
            break
    for order in route[1:-1]:
        if order > n_orders:
            continue
        else:
            sorted_indices.remove(order-1)
    routes.remove(route)

    deconstructed_route_list.append(route_1)
    # print('deconstructed_route_list',deconstructed_route_list)
    
    return routes, removed_list, deconstructed_route_list, sorted_indices

def find_next_list(primal_sorted_indices, routes, remaining_order = None):
    d = 0
    # print('routes:',routes)
    # print('primal_sorted_indices',primal_sorted_indices)
    i = primal_sorted_indices[d]
    next_order = i+1
    # print('next_order',next_order)
    if remaining_order != None:
        while next_order in remaining_order:
            i = primal_sorted_indices[d+1]
            next_order = i+1
    return [route for route in routes if next_order in route][0], next_order


def other_string_removal(d_matrix, routes, route, l_t, reference_order, n_orders, removed_list, deconstructed_route_list, primal_sorted_indices):
    distances = d_matrix[reference_order-1]
    sorted_indices = np.argsort(distances).tolist()
    route_1 = copy.deepcopy(route)
    a = 0
    for i in sorted_indices:
        if a <= l_t-1:
            if i+1 in route:
                route_1.remove(i+1)
                removed_list.append(i+1)
                route_1.remove(i+1+n_orders)
                removed_list.append(i+1+n_orders)
                a += 1
        else:
            break
    route_2 = copy.deepcopy(route)
    for order in route_2[1:-1]:
        if order > n_orders:
            continue
        else:
            primal_sorted_indices.remove(order-1)
    routes.remove(route)

    deconstructed_route_list.append(route_1)
    
    return routes, removed_list, deconstructed_route_list, primal_sorted_indices


In [21]:
# SISR removal function
def SISR_removal(L_max, avg_remove_order, routes, n_orders, d_matrix, remaining_orders = None):
    routes_copy = copy.deepcopy(routes)
    removed_list = []
    deconstructed_route_list = []
    k_s, l_s_max = number_of_strings(L_max, avg_remove_order, routes_copy, n_orders)
    k_s = min(k_s, len(routes_copy))
    for i in range(int(k_s)):
        if i == 0:
            start_order = int(order_to_start(n_orders))
            route = find_lists_containing_element(routes_copy, start_order)
            l_t = number_of_orders(l_s_max, route)
            routes_copy, removed_list, deconstructed_route_list, primal_sorted_indices = primal_string_removal(d_matrix, routes_copy, route, l_t, start_order, n_orders, removed_list, deconstructed_route_list)
        elif primal_sorted_indices == []:
            break
        else:
            route, next_order = find_next_list(primal_sorted_indices, routes_copy)
            l_t = number_of_orders(l_s_max, route)
            routes_copy, removed_list, deconstructed_route_list, primal_sorted_indices = other_string_removal(d_matrix, routes_copy, route, l_t, next_order, n_orders, removed_list, deconstructed_route_list, primal_sorted_indices)
   
    remaining_routes = deconstructed_route_list + routes_copy
    removed_list = removed_list #+ remaining_orders
    return remaining_routes, removed_list


In [22]:
import copy
import random
import numpy as np
import matplotlib.pyplot as plt

def SISR(seeding, n_orders, map_size, speed, extra_time, seed, I_max, penalty, t0, tf, L_max, c_avg):
    '''n_orders = 20, map_size = 3, speed = 4, extra_time = 10, seed = 1234, I_max = 1000, penalty = 2, t0 = 100, tf = 1, L_max = 10, c_avg = 10'''
    # 创建一个 PDPTWInstance 对象
    instance = PDPTWInstance(n=n_orders, map_size=map_size, speed=speed, extra_time=extra_time, seed = seed)
    
    random.seed(seeding)
    np.random.seed(seeding)
    
    df = instance.to_dataframe()
    d_last = []
    d_last.append([int(max(df.ID)+1),'d',df.X[0],df.Y[0],0,0,float('inf'),0,0])
    columns = ['ID', 'Type', 'X', 'Y', 'Demand', 'ReadyTime', 'DueDate', 'ServiceTime', 'PartnerID']
    d_last = pd.DataFrame(d_last, columns=columns)
    df = pd.concat([df, d_last], axis = 0)

    routes, remaining_orders = insertion_heuristics(df, num_vehicles = 4, battery = 25)


    # Full SISR process
    I_max = I_max
    locations = df[['X','Y']].values
    earliest = df[['ReadyTime']].values
    latest = df[['DueDate']].values
    demand = df[['Demand']].values
    services = df[['ServiceTime']].values
    services[0] = 0
    speed = speed
    penalty = penalty
    t0 = t0
    tf = tf
    
    df_orders = df.iloc[1:-1,:]
    stops = df_orders[['X','Y']].values
    e_times = df_orders[['ReadyTime']].values
    l_times = df_orders[['DueDate']].values
    
    def euclidean_distance(p1, p2):
        return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)
    
    d_matrix = []
    for i in range(n_orders):
        d_list = []
        for j in range(n_orders):
            d_1 = euclidean_distance(stops[i],stops[j]) # pickup-pickup
            t_1 = abs(e_times[i] - e_times[j])/60*speed
            dt_1 = d_1 + t_1*0.1
            
            d_2 = euclidean_distance(stops[i],stops[j+n_orders]) #pickup-dropoff
            t_2 = abs(e_times[i] - l_times[j+n_orders])/60*speed
            dt_2 = d_2 + t_2*0.1
            
            d_3 = euclidean_distance(stops[i+n_orders],stops[j]) #dropoff-pickup
            t_3 = abs(e_times[j] - l_times[i+n_orders])/60*speed
            dt_3 = d_3 + t_3*0.1
            
            d_4 = euclidean_distance(stops[i+n_orders],stops[j+n_orders]) #dropoff-dropoff
            t_4 = abs(e_times[j+n_orders] - l_times[i+n_orders])/60*speed
            dt_4 = d_4 + t_4*0.1
            
            d_min = min(dt_1, dt_2, dt_3, dt_4)[0]
            d_list.append(d_min)
        d_matrix.append(d_list)

    time_dict = {0:0}
    capacity_dict = {0:0}
    for route in routes:
        time_dict, capacity_dict = update_node(route, earliest, time_dict, capacity_dict, services, demand, locations, speed)
    cost = calculate_route_cost(routes, time_dict, locations, latest, penalty, remaining_orders)
    cost_best = cost
    saved_cost = cost
    best_routes = routes.copy()
    saved_routes = routes.copy()

    routes_1 = saved_routes.copy()

    iteration = 0

    for i in range(I_max):

        remaining_routes, removed_list = SISR_removal(L_max, c_avg, routes_1, n_orders, d_matrix)
        #Re-insertion
        insert_list = [[0]]
        for order in removed_list:
            if order <= n_orders:
                insert_list.append([order])
        routes_2, remaining_orders = insertion_heuristics(df, routes = remaining_routes, pickup_delivery_pairs = insert_list, battery = 25)

        time_dict = {0:0}
        capacity_dict = {0:0}
        for route in routes_2:
            time_dict, capacity_dict = update_node(route, earliest, time_dict, capacity_dict, services, demand, locations, speed)
        cost_new = calculate_route_cost(routes_2, time_dict, locations, latest, penalty, remaining_orders)

        routes_current = copy.deepcopy(routes_2)

        #simulated annealing
        if iteration == 0:
            t_now = t0
        else:
            t_now = t_update

        sa = t_now * np.log(np.random.uniform(0,1))

        if cost_new < saved_cost - sa:
            saved_routes = copy.deepcopy(routes_current)
            saved_cost = cost_new
            
        if cost_new < cost_best:
            best_routes = copy.deepcopy(routes_current)
            cost_best = cost_new 
        
        if iteration == 0:
            print(saved_cost, cost_best)

        c = (tf/t0)**(1/(iteration+1))
        t_update = c*t_now

        iteration += 1

        routes_1 = copy.deepcopy(saved_routes)
        # print(saved_cost, cost_best)
    return saved_cost, cost_best, saved_routes, best_routes

In [23]:
from joblib import Parallel, delayed

# Execute the function in parallel
p = 10
s_start = 1
results = Parallel(n_jobs=p)(delayed(SISR)(seeding = i, n_orders = 15, map_size = 2, speed = 4, extra_time = 0, seed = 1234, I_max = 1500, penalty = 2, t0 = 100, tf = 1, L_max = 10, c_avg = 10) for i in range(s_start,s_start+p+1))
for i in range(len(results)):
    print(results[i])

(56.15492475663809, 56.15492475663809, [[0, 2, 17, 6, 21, 0], [0, 12, 27, 15, 30, 9, 8, 23, 24, 0], [0, 4, 19, 7, 22, 0], [0, 1, 3, 11, 16, 18, 26, 10, 14, 5, 29, 13, 20, 25, 28, 0]], [[0, 12, 27, 15, 30, 9, 8, 23, 24, 0], [0, 2, 17, 6, 21, 0], [0, 4, 19, 7, 22, 0], [0, 1, 3, 11, 16, 18, 26, 10, 14, 5, 29, 13, 20, 25, 28, 0]])
(60.25406853782139, 60.25406853782139, [[0, 5, 20, 10, 25, 14, 4, 2, 29, 17, 19, 6, 21, 0], [0, 8, 23, 7, 22, 1, 13, 3, 11, 28, 12, 16, 18, 26, 27, 15, 30, 9, 24, 0], [0, 0], [0, 0]], [[0, 5, 20, 10, 25, 14, 4, 2, 29, 17, 19, 6, 21, 0], [0, 8, 23, 7, 22, 1, 13, 3, 11, 28, 12, 16, 18, 26, 27, 15, 30, 9, 24, 0], [0, 0], [0, 0]])
(66.57168192297316, 66.57168192297316, [[0, 2, 17, 7, 22, 5, 14, 4, 8, 29, 19, 23, 11, 20, 26, 10, 25, 0], [0, 13, 28, 12, 1, 3, 6, 27, 15, 16, 18, 30, 9, 24, 21, 0], [0, 0], [0, 0]], [[0, 2, 17, 7, 22, 5, 14, 4, 8, 29, 19, 23, 11, 20, 26, 10, 25, 0], [0, 13, 28, 12, 1, 3, 6, 27, 15, 16, 18, 30, 9, 24, 21, 0], [0, 0], [0, 0]])
(55.421912080

In [57]:
routes = [[0, 12, 27, 9, 15, 30, 24, 8, 23, 0], [0, 13, 28, 14, 10, 25, 5, 20, 4, 19, 29, 0], [0, 1, 3, 11, 26, 18, 16, 0], [0, 2, 17, 6, 7, 22, 21, 0]]

In [59]:
r_2 = 0
dist = 0
for route in routes_best:
    r_2 += len(route)**2
    dist += total_distance(route, locations)
r_2

388

In [50]:
routes_best = [[0, 12, 9, 27, 15, 30, 24, 0],
 [0, 13, 5, 28, 20, 10, 14, 25, 6, 29, 21, 0],
 [0, 4, 8, 19, 23, 0],
 [0, 2, 1, 3, 17, 7, 22, 11, 26, 18, 16, 0]]

In [51]:
time_dict = {0:0}
capacity_dict = {0:0}

instance = PDPTWInstance(n=15, map_size=2, speed=4, extra_time=0, seed = 1234)
df = instance.to_dataframe()
d_last = []
d_last.append([int(max(df.ID)+1),'d',df.X[0],df.Y[0],0,0,float('inf'),0,0])
columns = ['ID', 'Type', 'X', 'Y', 'Demand', 'ReadyTime', 'DueDate', 'ServiceTime', 'PartnerID']
d_last = pd.DataFrame(d_last, columns=columns)
df = pd.concat([df, d_last], axis = 0)

locations = df[['X','Y']].values
earliest = df[['ReadyTime']].values
latest = df[['DueDate']].values
demand = df[['Demand']].values
services = df[['ServiceTime']].values
services[0] = 0
speed = 4
penalty = 2
    
for route in routes_best:
    time_dict, capacity_dict = update_node(route, earliest, time_dict, capacity_dict, services, demand, locations, speed)

cost = 0
for vehicle_idx, route in enumerate(routes_best):
    for i in range(len(route) - 1):
        cost += euclidean_distance(locations[route[i]], locations[route[i+1]])
    for j in route:
        if time_dict[j] > latest[j]:
            cost += penalty  #penalty cost
# print(capacity_dict)

{0: 0, 12: 1, 9: 2, 27: 1, 15: 2, 30: 1, 24: 0, 13: 1, 5: 2, 28: 1, 20: 0, 10: 1, 14: 2, 25: 1, 6: 2, 29: 1, 21: 0, 4: 1, 8: 2, 19: 1, 23: 0, 2: 1, 1: 2, 3: 3, 17: 2, 7: 3, 22: 2, 11: 3, 26: 2, 18: 1, 16: 0}


In [52]:
dist = 0
for route in routes_best:
    dist += total_distance(route, locations)
print(dist)

38.48453854256509


In [85]:
instance = PDPTWInstance(n=15, map_size=2, speed=4, extra_time=0, seed = 1234)
df = instance.to_dataframe()
d_last = []
d_last.append([int(max(df.ID)+1),'d',df.X[0],df.Y[0],0,0,float('inf'),0,0])
columns = ['ID', 'Type', 'X', 'Y', 'Demand', 'ReadyTime', 'DueDate', 'ServiceTime', 'PartnerID']
d_last = pd.DataFrame(d_last, columns=columns)
df = pd.concat([df, d_last], axis = 0)

df_orders = df.iloc[1:-1,:]
stops = df_orders[['X','Y']].values
e_times = df_orders[['ReadyTime']].values
l_times = df_orders[['DueDate']].values

n_orders = 15
speed = 4

def euclidean_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

d_matrix = []
for i in range(n_orders):
    d_list = []
    for j in range(n_orders):
        d_1 = euclidean_distance(stops[i],stops[j]) # pickup-pickup
        t_1 = abs(e_times[i] - e_times[j])/60*speed
        dt_1 = d_1 + t_1*0
        
        d_2 = euclidean_distance(stops[i],stops[j+n_orders]) #pickup-dropoff
        t_2 = abs(e_times[i] - l_times[j+n_orders])/60*speed
        dt_2 = d_2 + t_2*0
        
        d_3 = euclidean_distance(stops[i+n_orders],stops[j]) #dropoff-pickup
        t_3 = abs(e_times[j] - l_times[i+n_orders])/60*speed
        dt_3 = d_3 + t_3*0
        
        d_4 = euclidean_distance(stops[i+n_orders],stops[j+n_orders]) #dropoff-dropoff
        t_4 = abs(e_times[j+n_orders] - l_times[i+n_orders])/60*speed
        dt_4 = d_4 + t_4*0
        
        d_min = min(dt_1, dt_2, dt_3, dt_4)[0]
        d_list.append(d_min)
    d_matrix.append(d_list)
d_matrix

[[0.0,
  0.5763308321920182,
  0.4976356025343404,
  1.7772625335432959,
  1.697137159539573,
  1.5046940787762277,
  1.6942702926309379,
  1.8316902655559708,
  0.9708379972190254,
  1.6121881803775546,
  1.3071962436890137,
  0.557064496286966,
  2.207887295730153,
  1.2325920371755639,
  0.6137839175326026],
 [0.5763308321920182,
  0.0,
  0.7197669946393361,
  0.3413642364364809,
  1.9576901474303512,
  1.1759092218065723,
  0.28110981757295106,
  1.3260755285448622,
  1.9546507868424212,
  2.673864562037499,
  1.4175200065967195,
  2.7762346037675463,
  1.643048118046852,
  0.6564839339143559,
  3.296338160652189],
 [0.4976356025343404,
  0.7197669946393361,
  0.0,
  0.6984437900773273,
  1.209820330281835,
  1.0510181348579724,
  0.7612867141960591,
  1.2361495949503878,
  1.0478320349943127,
  1.1236275244785408,
  0.8103823915702894,
  0.7475743051049799,
  1.9833201562064484,
  0.2867422524578328,
  0.5268236033669009],
 [1.7772625335432959,
  0.3413642364364809,
  0.6984437900

In [40]:
d_matrix[8]

[3.4375046638856923,
 3.33085365457784,
 2.81504450036747,
 1.9001040007578123,
 3.420382028551516,
 0.7015644853792445,
 1.890424455310149,
 1.1750412370851537,
 0.0,
 3.131922192708935,
 1.9495140624450618,
 2.1761395301532658,
 3.4560787957758943,
 2.8574280816893958,
 1.7877164780884405]

In [155]:
for i in range(1,31):
    if time_dict[i] > latest[i]:
        print(i)

16
19
20
29


In [25]:
df

,ID,Type,X,Y,Demand,ReadyTime,DueDate,ServiceTime,PartnerID
0,0,d,0.000000,0.000000,0,0,inf,9,0
1,1,cp,1.865814,-0.237070,1,5,inf,7,16
2,2,cp,1.757076,0.328910,1,49,inf,7,17
3,3,cp,1.065924,-1.052761,1,63,inf,8,18
4,4,cp,-0.615644,0.493126,1,74,inf,5,19
5,5,cp,-1.267637,-1.542348,1,60,inf,8,20
6,6,cp,1.859606,-1.741751,1,100,inf,9,21
7,7,cp,0.405854,-1.644285,1,111,inf,6,22
8,8,cp,0.225730,0.578537,1,112,inf,10,23
9,9,cp,-1.003392,1.734062,1,56,inf,9,24


In [27]:
euclidean_distance(locations[0], locations[12])

1.3634109565696748

In [31]:
total_distance(routes[3], locations)

11.58820283615022

In [22]:
time_dict

{0: 0,
 12: 37,
 9: 66.14209295229898,
 27: 81.44058672647415,
 15: 98,
 30: 114.249761783785,
 24: 139.81525950697858,
 13: 40.87451778071952,
 5: 110.5156942308033,
 28: 130.24653995906715,
 20: 152.51737637339787,
 10: 160.82882417294945,
 14: 182.76120794515595,
 25: 195.9958961236746,
 6: 263.739072141848,
 29: 284.4054224510603,
 21: 314.92927619335785,
 4: 74,
 8: 112,
 19: 151.97727069226374,
 23: 176.78207491685765,
 2: 49,
 1: 64.64496248288027,
 3: 88.78161372914943,
 17: 107.57811864873946,
 7: 119.79476591233373,
 22: 136.71762043323403,
 11: 151.7944777134285,
 26: 186.90147430938708,
 18: 209.05721018294142,
 16: 224.52174422095652}

In [46]:
for i in time_dict:
    if time_dict[i] > latest[i]:
        print(i)

16
19
29
21


In [ ]:
import numpy as np
import copy

class SISR:
    def __init__(self, L_max, avg_remove_order, routes, n_orders, d_matrix, remaining_orders=None):
        self.L_max = L_max
        self.avg_remove_order = avg_remove_order
        self.routes = routes
        self.n_orders = n_orders
        self.d_matrix = d_matrix
        self.remaining_orders = remaining_orders
        self.removed_list = []
        self.deconstructed_route_list = []

    def number_of_strings(self):
        T_avg = np.floor(self.n_orders / len(self.routes))
        l_s_max = min(T_avg, self.L_max)
        k_s_max = 4 * self.avg_remove_order / (1 + l_s_max) - 1
        k_s = np.floor(np.random.uniform(1, k_s_max + 1))
        return k_s, l_s_max

    def number_of_orders(self, l_s_max, route):
        l_t_max = min(l_s_max, (len(route) - 2) / 2)
        l_t = np.floor(np.random.uniform(1, l_t_max + 1))
        return l_t

    def order_to_start(self, remaining_order=None):
        reference_order = int(np.floor(np.random.uniform(1, self.n_orders + 1)))
        if remaining_order:
            while reference_order in remaining_order:
                reference_order = int(np.floor(np.random.uniform(1, self.n_orders + 1)))
        return reference_order

    def find_lists_containing_element(self, order):
        return [route for route in self.routes if order in route][0]

    def remove_orders_from_route(self, route, l_t, reference_order):
        distances = self.d_matrix[reference_order - 1]
        sorted_indices = np.argsort(distances).tolist()
        route_copy = copy.deepcopy(route)
        a = 0
        for i in sorted_indices:
            if a >= l_t:
                break
            order = i + 1
            if order in route_copy:
                route_copy.remove(order)
                self.removed_list.append(order)
                paired_order = order + self.n_orders
                if paired_order in route_copy:
                    route_copy.remove(paired_order)
                    self.removed_list.append(paired_order)
                a += 1
        return route_copy, sorted_indices

    def primal_string_removal(self, route, l_t, reference_order):
        route_1, sorted_indices = self.remove_orders_from_route(route, l_t, reference_order)
        for order in route[1:-1]:
            if order <= self.n_orders:
                sorted_indices.remove(order - 1)
        self.routes.remove(route)
        self.deconstructed_route_list.append(route_1)
        return sorted_indices

    def find_next_list(self, primal_sorted_indices, remaining_order=None):
        for i in primal_sorted_indices:
            next_order = i + 1
            if remaining_order and next_order in remaining_order:
                continue
            return [route for route in self.routes if next_order in route][0], next_order

    def other_string_removal(self, route, l_t, reference_order, primal_sorted_indices):
        route_1, sorted_indices = self.remove_orders_from_route(route, l_t, reference_order)
        for order in route[1:-1]:
            if order <= self.n_orders:
                primal_sorted_indices.remove(order - 1)
        self.routes.remove(route)
        self.deconstructed_route_list.append(route_1)
        return primal_sorted_indices

    def SISR_removal(self):
        routes_copy = copy.deepcopy(self.routes)
        k_s, l_s_max = self.number_of_strings()
        k_s = min(k_s, len(routes_copy))
        
        for i in range(int(k_s)):
            if i == 0:
                start_order = int(self.order_to_start())
                route = self.find_lists_containing_element(start_order)
                l_t = self.number_of_orders(l_s_max, route)
                primal_sorted_indices = self.primal_string_removal(route, l_t, start_order)
            else:
                route, next_order = self.find_next_list(primal_sorted_indices)
                l_t = self.number_of_orders(l_s_max, route)
                primal_sorted_indices = self.other_string_removal(route, l_t, next_order, primal_sorted_indices)
        
        remaining_routes = self.deconstructed_route_list + routes_copy
        
        # Count the total number of elements in remaining_routes
        total_elements_in_remaining_routes = sum(len(route) for route in remaining_routes)
        
        # Check if the number of elements in removed_list and remaining_routes equals n_orders
        if len(self.removed_list) + total_elements_in_remaining_routes != self.n_orders:
            self.removed_list = self.removed_list + self.remaining_orders if self.remaining_orders else self.removed_list
        
        return remaining_routes, self.removed_list
